In [29]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import pandas as pd
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import math
import re

# the following are NLTK and WordNet packages for the baseline method:
#from __future__ import unicode_literals
from nltk.corpus.reader.wordnet import NOUN
from nltk.corpus import wordnet as wn
from nltk.compat import python_2_unicode_compatible
from nltk.stem import WordNetLemmatizer
import nltk

%matplotlib inline

In [30]:
# the following is based on the source code provided for WordNet Lemmatizer (see report)
class WordNetLemmatizer(object):
    def __init__(self):
        pass
    
    def lemmatize(self, word, pos=NOUN):
        lemmas = wn._morphy(word, pos)
        return min(lemmas, key=len) if lemmas else word
    
    def __rept__(self):
        return '<WordNetLemmatizer>'
    
    # unload wordnet
    def teardown_module(module=None):
        from nltk.corpus import wordnet
        wn._unload()

In [31]:
# test wordnet lemmatizer
wnl = WordNetLemmatizer()

In [44]:
# create mappings between Penn treebank and WordNet POS tags:
tag_map = {
        'CC':None, # coordin. conjunction (and, but, or)  
        'CD':wn.NOUN, # cardinal number (one, two)             
        'DT':None, # determiner (a, the)                    
        'EX':wn.ADV, # existential ‘there’ (there)           
        'FW':None, # foreign word (mea culpa)             
        'IN':wn.ADV, # preposition/sub-conj (of, in, by)   
        'JJ':wn.ADJ, # adjective (yellow)                  
        'JJR':wn.ADJ, # adj., comparative (bigger)          
        'JJS':wn.ADJ, # adj., superlative (wildest)           
        'LS':None, # list item marker (1, 2, One)          
        'MD':None, # modal (can, should)                    
        'NN':wn.NOUN, # noun, sing. or mass (llama)          
        'NNS':wn.NOUN, # noun, plural (llamas)                  
        'NNP':wn.NOUN, # proper noun, sing. (IBM)              
        'NNPS':wn.NOUN, # proper noun, plural (Carolinas)
        'PDT':wn.ADJ, # predeterminer (all, both)            
        'POS':None, # possessive ending (’s )               
        'PRP':None, # personal pronoun (I, you, he)     
        'PRP$':None, # possessive pronoun (your, one’s)    
        'RB':wn.ADV, # adverb (quickly, never)            
        'RBR':wn.ADV, # adverb, comparative (faster)        
        'RBS':wn.ADV, # adverb, superlative (fastest)     
        'RP':wn.ADJ, # particle (up, off)
        'SYM':None, # symbol (+,%, &)
        'TO':None, # “to” (to)
        'UH':None, # interjection (ah, oops)
        'VB':wn.VERB, # verb base form (eat)
        'VBD':wn.VERB, # verb past tense (ate)
        'VBG':wn.VERB, # verb gerund (eating)
        'VBN':wn.VERB, # verb past participle (eaten)
        'VBP':wn.VERB, # verb non-3sg pres (eat)
        'VBZ':wn.VERB, # verb 3sg pres (eats)
        'WDT':None, # wh-determiner (which, that)
        'WP':None, # wh-pronoun (what, who)
        'WP$':None, # possessive (wh- whose)
        'WRB':None, # wh-adverb (how, where)
        '$':None, #  dollar sign ($)
        '#':None, # pound sign (#)
        '“':None, # left quote (‘ or “)
        '”':None, # right quote (’ or ”)
        '(':None, # left parenthesis ([, (, {, <)
        ')':None, # right parenthesis (], ), }, >)
        ',':None, # comma (,)
        '.':None, # sentence-final punc (. ! ?)
        ':':None # mid-sentence punc (: ; ... – -)
    }

In [11]:
# load data
charts = pd.read_csv("NOTEEVENTS.csv", dtype={"ROW_ID":"int64", "CHARTTIME":"str", "STORETIME":"str"})

In [12]:
#charts
notes_df = charts[['TEXT']]


In [55]:
# preprocessing: for patients 1 through "limit," split notes by token (on newline and space characters) to create a list of lists
# additional preprocessing: remove tokens that contain a digit
limit = 500
notes = []
for note in notes_df['TEXT'][:limit]:
    notes.append([token for token in re.split(r'[\n ]', note) if token != '' and not bool(re.search(r'\d', token)) and not bool(re.search(r'\W', token))])


In [56]:
unchanged = 0
changed = 0
for item in notes:
    tagged = nltk.pos_tag(item)
    for tup in tagged:
        if tag_map[tup[1]] is None:
            # skip words that don't have a POS tag in WordNet(conjunctions, etc.)
            continue
        #print(tup[0],tag_map[tup[1]])
        if wnl.lemmatize(tup[0], pos=tag_map[tup[1]])==tup[0]:
            unchanged+=1
        else:
            changed+=1

print('Unchanged: '+str(unchanged)+'\nChanged: '+str(changed))


Unchanged: 367225
Changed: 79656
